In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from tools.notebooks import env

context = Context()

In [ ]:
from contracts.staking_contract import StakingContract, EnterFarmEvent, ExitFarmEvent, ClaimRewardsFarmEvent
from utils.contract_data_fetchers import StakingContractDataFetcher
staking_contract: StakingContract
staking_contract = context.get_contracts(config.STAKINGS_V2)[0]

In [ ]:
from trackers.staking_economics_tracking import StakingEconomics

staking_tracker = StakingEconomics(staking_contract.address, context.network_provider)

In [ ]:
from contracts.metastaking_contract import MetaStakingContract, EnterMetastakeEvent, ExitMetastakeEvent, ClaimRewardsMetastakeEvent
from utils.contract_data_fetchers import MetaStakingContractDataFetcher
metastaking_contract: MetaStakingContract
metastaking_contract = context.get_contracts(config.METASTAKINGS_BOOSTED)[0]

In [ ]:
ride_purse = Address('')
depositer = Account(pem_file=env.USER1_PEM)
depositer.address = ride_purse
depositer.sync_nonce(context.network_provider.proxy)

upgrade

In [ ]:
from contracts.staking_contract import StakingContractVersion
# upgrade all farms
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[StakingContract] = context.get_contracts(config.STAKINGS_V2)
for contract in contracts:
    contract.version = StakingContractVersion.V2
    tx_hash = contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         config.STAKING_V3_BYTECODE_PATH, [], no_init=True)
    context.network_provider.check_complex_tx_status(tx_hash, "staking upgrade")

contract config

In [ ]:
from time import sleep
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[StakingContract] = context.get_contracts(config.STAKINGS_V2)
simple_lock_energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
for contract in contracts:
    tx_hash = contract.set_boosted_yields_rewards_percentage(context.deployer_account, context.network_provider.proxy, 6000)
    tx_hash = contract.set_boosted_yields_factors(context.deployer_account, context.network_provider.proxy, 
                                                  [2, 1, 0, 1, 1])
    tx_hash = contract.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)
    tx_hash = simple_lock_energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, contract.address)
    sleep(2)

In [ ]:
# resume all farms
context.deployer_account.sync_nonce(context.network_provider.proxy)
contracts : list[StakingContract] = context.get_contracts(config.STAKINGS_V2)
for contract in contracts:
    tx_hash = contract.resume(context.deployer_account, context.network_provider.proxy)

transfer tokens and prepare accounts

In [ ]:
staking_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, 1000000000)

In [ ]:
staking_contract.add_admin(context.deployer_account, context.network_provider.proxy, depositer.address.bech32())
staking_contract.topup_rewards(depositer, context.network_provider.proxy, 10000000000000000000)

In [ ]:
staking_contract.set_max_apr(context.deployer_account, context.network_provider.proxy, 10000)

In [ ]:
staking_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)

In [ ]:
txhash = staking_contract.register_farm_token(context.deployer_account, context.network_provider.proxy, ["STKTST", "STKTST"])
status = context.network_provider.check_simple_tx_status(txhash, "set register farm token")

In [ ]:
status = context.network_provider.check_simple_tx_status("5f5e3b111bc5ca2f59e4f8bb8faf3aa7e7827bc0247808b53eb7bd58e9db19e4", "set boosted yields factors in farm")

In [ ]:
farm_token_hex = MetaStakingContractDataFetcher(Address(metastaking_contract.address), context.network_provider.proxy.url).get_data("getDualYieldTokenId")
print(hex_to_string(farm_token_hex))

enter stake

In [ ]:
event = EnterFarmEvent(staking_contract.farming_token, 0, 1000000000000000000, "", 0, 0)
txhash = staking_contract.stake_farm(context.network_provider, depositer, event, True)

claim stake

In [ ]:
event = ClaimRewardsFarmEvent(100000000000000000, 10, "")
txhash = staking_contract.claim_rewards(context.network_provider, depositer, event)

In [ ]:
from utils.utils_chain import get_token_details_for_address

farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(staking_contract.farm_token, depositer.address.bech32(), context.network_provider.proxy)
event = ClaimRewardsFarmEvent(farm_token_amount, farm_token_nonce, "")
txhash = staking_contract.claim_rewards(context.network_provider, depositer, event)

exit stake

In [ ]:
event = ExitFarmEvent(staking_contract.farm_token, 1000000, 9, "", 1000000)
txhash = staking_contract.unstake_farm(context.network_provider, depositer, event)

In [ ]:
from utils.utils_chain import get_token_details_for_address

farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(staking_contract.farm_token, depositer.address.bech32(), context.network_provider)
event = ClaimRewardsFarmEvent(farm_token_amount, farm_token_nonce, "")
txhash = staking_contract.claim_rewards(context.network_provider, depositer, event)

report contract data

In [ ]:
staking_tracker.update_data()
staking_tracker.report_current_tracking_data()

Shadowfork control

In [ ]:
from contracts.builtin_contracts import SFControlContract

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
sf_control_contract.epoch_fast_forward(context.deployer_account, context.network_provider.proxy, 16, 20)